### Setup and imports
Let us first get our imports out of the way:

In [1]:
from pydrake.all import *

from manipulation.station import (
    LoadScenario,
    MakeHardwareStation,
)

from pathlib import Path
import numpy as np

### Meshcat Visualization

As always, let's start Meshcat for our 3D visualization!

In [2]:
# Start meshcat for visualization
meshcat = StartMeshcat()
print("Click the link above to open Meshcat in your browser!")

INFO:drake:Meshcat listening for connections at http://localhost:7000


Click the link above to open Meshcat in your browser!


In [3]:
abs_table_sdf_path = f"{Path.cwd()}/assets/table.sdf"
with open(abs_table_sdf_path, "r") as fin:
    table_sdf = fin.read()

scenario_yaml = f"""directives:
- add_model:
    name: iiwa
    file: package://drake_models/iiwa_description/sdf/iiwa7_no_collision.sdf
- add_weld:
    parent: world
    child: iiwa::iiwa_link_0
- add_model:
    name: wsg
    file: package://manipulation/hydro/schunk_wsg_50_with_tip.sdf
- add_weld:
    parent: iiwa::iiwa_link_7
    child: wsg::body
    X_PC:
        translation: [0, 0, 0.09]
        rotation: !Rpy {{ deg: [90, 0, 90]}}
- add_model:
    name: table
    file: file://{abs_table_sdf_path}
- add_weld:
    parent: world
    child: table::table_link
    X_PC:
        translation: [0.0, 0.0, -0.05]
        rotation: !Rpy {{ deg: [0, 0, -90] }}

- add_model:
    name: ball0
    file: package://drake_models/manipulation_station/sphere.sdf
    default_free_body_pose:
        base_link:
            translation: [0, -0.5, 0.02]
            rotation: !Rpy {{ deg: [0, 0, 0] }}

model_drivers:
    iiwa: !IiwaDriver
      control_mode: position_only
      hand_model_name: wsg
    wsg: !SchunkWsgDriver {{}}
"""

In [ ]:
def move_arm_keyframes(keyframes, times, gripper_positions=None):
    """
    Move arm through keyframes (joint positions) with optional gripper control.
    
    Args:
        keyframes: List of 7-DOF joint positions
        times: List of times for each keyframe
        gripper_positions: List of gripper positions (optional)
    """
    # Create trajectories
    q_traj = PiecewisePolynomial.CubicShapePreserving(
        times, 
        np.column_stack(keyframes),
        zero_end_point_derivatives=True
    )
    v_traj = q_traj.MakeDerivative()
    
    # Build system
    scenario = LoadScenario(data=scenario_yaml)
    station = MakeHardwareStation(scenario, meshcat=meshcat)
    builder = DiagramBuilder()
    builder.AddSystem(station)
    
    # Velocity control
    v_source = builder.AddSystem(TrajectorySource(v_traj))
    integrator = builder.AddSystem(Integrator(7))
    builder.Connect(
        v_source.get_output_port(), 
        integrator.get_input_port()
    )
    builder.Connect(
        integrator.get_output_port(), 
        station.GetInputPort("iiwa.position")
    )
    
    # Gripper control
    if gripper_positions is not None:
        wsg_traj = PiecewisePolynomial.FirstOrderHold(
            times, 
            np.array([gripper_positions])
        )
        wsg_source = builder.AddSystem(TrajectorySource(wsg_traj))
        builder.Connect(
            wsg_source.get_output_port(), 
            station.GetInputPort("wsg.position")
        )
    else:
        wsg_command = builder.AddSystem(ConstantVectorSource([0.1]))
        builder.Connect(
            wsg_command.get_output_port(), 
            station.GetInputPort("wsg.position")
        )
    
    diagram = builder.Build()
    
    # Create simulator
    simulator = Simulator(diagram)
    context = simulator.get_mutable_context()
    integrator.set_integral_value(
        integrator.GetMyContextFromRoot(context), 
        keyframes[0]
    )
    simulator.set_target_realtime_rate(1.0)
    
    return simulator

# Example usage:
keyframes = [
    np.zeros(7),
    np.array([-1.57, 0.1, 0, 1.2, 0, 1.6, 0]),
]
times = [0, 3.0]
simulator = move_arm_keyframes(keyframes, times)
simulator.AdvanceTo(3.0)